In [ ]:
#My path
path = '/Users/claes/Documents/Mine dokumenter/Universitet/7.semester/Deep_learning/Final project/data/dataset/Revco/'

#Your path
#path = '/Users/claes/Desktop/Deep_learning/Week_6'

## Lægge alle data fra hver fryser sammen, klar til autoencoder

In [3]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler

# Step 1: Load all parquet files matching the pattern
all_files = glob.glob(path + "cleaned_data_*.parquet")
#all_files = glob.glob(path + "cleaned_data_806016.parquet")

# Step 2: Define the columns to drop (conditionally)
drop_columns_set_1 = ['Datetime', 'State', 'Type', 'Event']
drop_columns_set_2 = drop_columns_set_1 + ['main_fault']

# Step 3: Initialize a list to store dataframes
dfs = []

# Step 4: Load, preprocess, and concatenate data
for file in all_files:
    df = pd.read_parquet(file)
    
    # Drop columns that exist in the dataframe
    if 'main_fault' in df.columns:
        df.drop(columns=[col for col in drop_columns_set_2 if col in df.columns], inplace=True)
    else:
        df.drop(columns=[col for col in drop_columns_set_1 if col in df.columns], inplace=True)
    
    # Append the processed dataframe to the list
    dfs.append(df)

# Step 5: Combine all dataframes into one
combined_df = pd.concat(dfs, axis=0, ignore_index=True)

# Step 6: Normalize the combined dataset
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(combined_df)

# Convert back to a DataFrame for easier handling (optional)
normalized_df = pd.DataFrame(data_normalized, columns=combined_df.columns)

normalized_df.to_parquet(path + 'combined_cleaned_data.parquet', index=False)


In [4]:
print(f"Længde af combined df: {normalized_df.shape[0]:,}")

Længde af combined df: 73,276,975


In [ ]:
import pandas as pd
import glob
import time
from sklearn.preprocessing import MinMaxScaler

start_time = time.time()  # Track total time

# Step 1: Load all parquet files matching the pattern
all_files = glob.glob(path + "cleaned_data_*.parquet")

for file in all_files:
    print(file)

# Step 2: Define the columns to drop (conditionally)
#drop_columns_set_1 = ['Datetime', 'State', 'Type', 'Event']
drop_columns_set_1 = ['State', 'Type', 'Event']
drop_columns_set_2 = drop_columns_set_1 + ['main_fault']

# Step 3: Initialize a list to store dataframes
dfs = []

# Step 4: Load, preprocess, and concatenate data
for file in all_files:
    df = pd.read_parquet(file)
    
    # Drop columns that exist in the dataframe
    if 'main_fault' in df.columns:
        df.drop(columns=[col for col in drop_columns_set_2 if col in df.columns], inplace=True)
    else:
        df.drop(columns=[col for col in drop_columns_set_1 if col in df.columns], inplace=True)
    
    # Append the processed dataframe to the list
    dfs.append(df)


# Step 5: Combine all dataframes into one
combined_df = pd.concat(dfs, axis=0, ignore_index=True)

print("step5")
print(combined_df.head(20))

elapsed_time = time.time() - start_time
print(f"Step 5 time: {elapsed_time:.2f} seconds")

# Step 6: Sort by Datetime and ensure it is in datetime format
combined_df['Datetime'] = pd.to_datetime(combined_df['Datetime'])
#combined_df.sort_values(by='Datetime', inplace=True)

print("step6")
print(combined_df.head(20))

elapsed_time = time.time() - start_time
print(f"Step 6 time: {elapsed_time:.2f} seconds")

# Step 7: Identify continuous sequences
# Calculate time difference in seconds
combined_df['time_diff'] = combined_df['Datetime'].diff().dt.total_seconds()

print("step7.1")
print(combined_df.head(20))

# Identify the start of a new sequence
combined_df['is_continuous'] = (combined_df['time_diff'].between(59, 61)) | (combined_df.index == 0)

print("step7.2")
print(combined_df.head(20))

# Assign sequence groups
combined_df['sequence_group'] = (~combined_df['is_continuous']).cumsum()

print("step7.3")
print(combined_df.head(20))

# Step 8: Filter only sequences of at least 100 continuous rows
valid_sequences = combined_df.groupby('sequence_group').filter(lambda x: len(x) >= 100)

print(valid_sequences)

elapsed_time = time.time() - start_time
print(f"Step 8 time: {elapsed_time:.2f} seconds")

# Step 9: Create time-windowed and flattened data
time_windows = []
window_size = 10

for _, group in valid_sequences.groupby('sequence_group'):
    for start_idx in range(0, len(group) - window_size + 1):
        # Extract the time window
        window = group.iloc[start_idx:start_idx + window_size].drop(['Datetime', 'time_diff', 'is_continuous', 'sequence_group'], axis=1)
        
        # Flatten the window
        flattened_window = window.values.flatten()  # Converts 2D array (100 x num_features) into 1D array
        
        # Append the flattened window to the list
        time_windows.append(flattened_window)
    print(group)

elapsed_time = time.time() - start_time
print(f"Step 9 time: {elapsed_time:.2f} seconds")

print(time_windows)

# Convert the list of flattened windows into a DataFrame for easier handling
flattened_df = pd.DataFrame(time_windows)

# Optional: Save the flattened data for later use
flattened_df.to_csv('flattened_time_windows.csv', index=False)

In [ ]:
elapsed_time = time.time() - start_time
print(f"Step 9 time: {elapsed_time:.2f} seconds")

# Step 10: Normalize each window
scaler = MinMaxScaler()
time_windows_normalized = [scaler.fit_transform(window) for window in time_windows]

elapsed_time = time.time() - start_time
print(f"Step 10 time: {elapsed_time:.2f} seconds")

# Step 11: Save the prepared data (optional)
# Convert the time-windowed data back to a DataFrame if needed
# Note: Adjust indices or format to save them meaningfully
prepared_data = pd.DataFrame({
    'time_window': time_windows_normalized
})

prepared_data.to_parquet(path + 'time_windowed_cleaned_data.parquet', index=False)

In [4]:
time_windows_normalized.shape

AttributeError: 'list' object has no attribute 'shape'

## 